In [33]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import csv

In [34]:
click_data = pd.read_csv("../data/click_data.csv")
emotional_events = pd.read_csv("../data/emotional_events.csv")
messages_data = pd.read_csv("../data/messages_data.csv")
user_information = pd.read_csv("../data/user_information.csv")
new_features = pd.read_csv("../feature_Engineering/new_features.csv")

In [35]:
#This columns are required in the new_feature File

#userId,task,number_interactions_with_assistant,spent_time_with_assistant (seconds),number_interactions_with_tables,spent_time_with_tables (seconds),confidence,arousal,valence,understanding,task_time (seconds)

### From message_data we are able to extract the number of interactions with the assistand and how long an user takes at a task

In [36]:
messages_data.sort_values(by=["userId", "task", "timestamp_unix"], inplace=True)
messages_data.reset_index(drop=True, inplace=True)

messages_data["number_interactions_with_assistant"] = 0
messages_data["spent_time_with_assistant (seconds)"] = 0

for (userId, task), group in messages_data.groupby(['userId', 'task']):
    number_interactions = group.shape[0]
    
    first_interaction_time = group['timestamp_unix'].min()
    last_interaction_time = group['timestamp_unix'].max()
    
    total_time_spent = last_interaction_time - first_interaction_time
    
    messages_data.loc[group.index, 'number_interactions_with_assistant'] = number_interactions
    messages_data.loc[group.index, 'spent_time_with_assistant (seconds)'] = total_time_spent

    with open('../feature_Engineering/new_features.csv', 'a') as file:
        file.write(f"{userId},{task},{number_interactions},{total_time_spent}\n")


### Exploring click_data, able to extract interactions at a task

In [37]:
click_data.sort_values(by=["userId", "task"], inplace=True)
click_data.reset_index(drop=True, inplace=True)

click_data["number_interactions_with_tables"] = 0
click_data["spent_time_with_tables (seconds)"] = 0

for (userId, task), group in click_data.groupby(['userId', 'task']):
    number_interactions = group.shape[0]
    
    first_interaction_time = group['timestamp_unix'].min()
    last_interaction_time = group['timestamp_unix'].max()
    
    total_time_spent = last_interaction_time - first_interaction_time
    
    click_data.loc[group.index, 'number_interactions_with_tables'] = number_interactions
    click_data.loc[group.index, 'spent_time_with_tables (seconds)'] = total_time_spent
    
    rows = []
    csv_filename = '../feature_Engineering/new_features.csv'
    # Read existing data from the new_features file
    with open(csv_filename, 'r') as file:
        reader = csv.DictReader(file)
        rows = list(reader)

    # Check if the row exists and update it
    row_exists = False
    for row in rows:
        user_feature = int(row['userId'])
        task_feature = int(row['task'])
        userID = int(userId)
        task = int(task)
        if user_feature == userId and task_feature == task:
            row['number_interactions_with_tables'] = number_interactions
            row['spent_time_with_tables (seconds)'] = total_time_spent
            row_exists = True
            break

    # If the row doesn't exist, append a new row
    if not row_exists:
        new_row = {
            'userId': userId,
            'task': task,
            'number_interactions_with_assistant': 0,
            'spent_time_with_assistant (seconds)': 0,
            'number_interactions_with_tables': number_interactions,
            'spent_time_with_tables (seconds)': total_time_spent
        }
        rows.append(new_row)

    # Write the updated data back to the CSV file after each iteration
    with open(csv_filename, 'w', newline='') as file:
        fieldnames = ['userId', 'task', 'number_interactions_with_assistant', 'spent_time_with_assistant (seconds)',
                        'number_interactions_with_tables', 'spent_time_with_tables (seconds)', 'confidence', 'arousal', 'valence', 'understanding', 'task_time (seconds)']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

### Exploring emotional_events, able to extract time per task, confidence, understanding, arousal and valence

In [38]:
# Filter the DataFrame to keep only the rows where 'task_type' is 'Information_finding'
filtered_df_information_finding = emotional_events[emotional_events['task_type'] == 'information_finding']
filtered_df_information_finding['arousal'].replace('AO07', '7', inplace=True)
filtered_df_information_finding['valence'].replace('AO07', '7', inplace=True)

for index, row in filtered_df_information_finding.iterrows():
    if row[['confidence', 'arousal', 'understanding', 'valence']].notnull().all():
        userId = row['userId']
        task = row['task']
        confidence = int(row['confidence'])
        arousal = int(row['arousal'])
        understanding = int(row['understanding'])
        valence = int(row['valence'])
        task_time = int(row['task_time'] )
        
        rows = []
        csv_filename = '../feature_Engineering/new_features.csv'
        # Read existing data from the new_features file
        with open(csv_filename, 'r') as file:
            reader = csv.DictReader(file)
            rows = list(reader)

        # Check if the row exists and update it
        row_exists = False
        for row in rows:
            user_feature = int(row['userId'])
            task_feature = int(row['task'])
            if user_feature == userId and task_feature == task:
                row['confidence'] = confidence
                row['arousal'] = arousal
                row['valence'] = valence
                row['understanding'] = understanding
                row['task_time (seconds)'] = task_time
                row_exists = True
                break

        # If the row doesn't exist, append a new row
        if not row_exists:
            new_row = {
                'userId': userId,
                'task': task,
                'number_interactions_with_assistant': 0,
                'spent_time_with_assistant (seconds)': 0,
                'number_interactions_with_tables': 0,
                'spent_time_with_tables (seconds)': 0,
                'confidence': confidence,
                'arousal': arousal,
                'valence': valence,
                'understanding': understanding,
                'task_time (seconds)': task_time
            }
            rows.append(new_row)

        # Write the updated data back to the CSV file after each iteration
        with open(csv_filename, 'w', newline='') as file:
            fieldnames = ['userId', 'task', 'number_interactions_with_assistant', 'spent_time_with_assistant (seconds)',
                        'number_interactions_with_tables', 'spent_time_with_tables (seconds)', 'confidence', 'arousal', 'valence', 'understanding', 'task_time (seconds)']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)


/var/folders/8v/x1s0jcnx06v5nb267srbbnnm0000gn/T/ipykernel_8891/2591562298.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df_information_finding['arousal'].replace('AO07', '7', inplace=True)
/var/folders/8v/x1s0jcnx06v5nb267srbbnnm0000gn/T/ipykernel_8891/2591562298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_i